#Description

I have used gpt2-medium model for this project. The dataset which i have used is wikitext-2-v1 which is quite small dataset to train model on. For this reason the model's accuracy is not good. Previously I tried to train model with wikitext-103-v1 dataset a huge dataset and it was very difficult to train model with that dataset and a T4 gpu. Even 1 epoch took 4 hours to complete. This is why I used wikitext-2-v1 dataset.

#Installing Libraries

In [1]:
!pip install transformers datasets accelerate
!pip install -U datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

#Loading Dataset

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

#Tokenizing and Training

In [3]:
model_name = "gpt2-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens["input_ids"] = [list(map(int, ids)) for ids in tokens["input_ids"]]
    tokens["attention_mask"] = [list(map(int, mask)) for mask in tokens["attention_mask"]]
    return tokens

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

def filter_empty(example):
    return any(tok != tokenizer.pad_token_id for tok in example["input_ids"])

tokenized = tokenized.filter(filter_empty)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

trainer.save_model("./results")
tokenizer.save_pretrained("./results")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

Filter:   0%|          | 0/36718 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3760 [00:00<?, ? examples/s]

/tmp/ipython-input-3-3921379601.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.133400,3.158055
2,2.999600,3.140224
3,2.949600,3.138185


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/merges.txt',
 './results/added_tokens.json')

#Evaluating the model

In [4]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_path = "./results"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_next_word(prompt, top_k=5):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        next_token_logits = outputs.logits[0, -1, :]

    top_k_probs, top_k_indices = torch.topk(next_token_logits, top_k)
    top_k_tokens = [tokenizer.decode([token_id]).strip() for token_id in top_k_indices]

    print(f"\nPrompt: {prompt}")
    print(f"Top-{top_k} Next Token Predictions:")
    for i, (token, score) in enumerate(zip(top_k_tokens, top_k_probs.tolist()), 1):
        print(f"{i}. {token} (logit: {score:.2f})")

predict_next_word("The sky is")
predict_next_word("Artificial intelligence will")
predict_next_word("He opened the")



Prompt: The sky is
Top-5 Next Token Predictions:
1. blue (logit: -79.88)
2. a (logit: -80.43)
3. the (logit: -80.71)
4. full (logit: -81.53)
5. dark (logit: -81.60)

Prompt: Artificial intelligence will
Top-5 Next Token Predictions:
1. be (logit: -112.05)
2. become (logit: -113.63)
3. not (logit: -113.95)
4. have (logit: -113.98)
5. also (logit: -114.20)

Prompt: He opened the
Top-5 Next Token Predictions:
1. door (logit: -105.59)
2. scoring (logit: -105.60)
3. season (logit: -106.18)
4. game (logit: -107.05)
5. match (logit: -107.21)


In [5]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch
from tqdm import tqdm

model_path = "./results"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path).eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

raw_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")

def evaluate_top_k(model, dataset, tokenizer, k=5, max_samples=1000, max_length=128):
    topk_correct = 0
    total = 0

    for example in tqdm(dataset.select(range(min(max_samples, len(dataset))))):
        text = example["text"].strip()
        if not text:
            continue

        tokens = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        )
        input_ids = tokens["input_ids"].squeeze(0).to(device)
        attention_mask = tokens["attention_mask"].squeeze(0).to(device)

        if input_ids.size(0) < 2:
            continue

        with torch.no_grad():
            outputs = model(input_ids=input_ids[:-1].unsqueeze(0), attention_mask=attention_mask[:-1].unsqueeze(0))
            logits = outputs.logits.squeeze(0)

        for i in range(len(input_ids) - 1):
            next_token = input_ids[i + 1].item()
            top_k_preds = torch.topk(logits[i], k).indices.tolist()
            if next_token in top_k_preds:
                topk_correct += 1
            total += 1

    acc = topk_correct / total if total > 0 else 0.0
    print(f"\n Top-{k} Accuracy: {acc:.4f} ({topk_correct}/{total})")
    return acc

evaluate_top_k(model, raw_dataset, tokenizer, k=5, max_samples=1000)


100%|██████████| 1000/1000 [00:31<00:00, 31.63it/s]


 Top-5 Accuracy: 0.3336 (27285/81788)


0.33360639702645867